锚框

以每个像素为中心生成多个大小和宽高比不同的边界框（锚框）
   预测每个锚框内是否还有关注的物体，如果是，则预测从这个锚框到真实边缘框的偏移

IoU——交并比
用于计算两个框之间的相似度
Jaccard系数：J(A,B) = 交/并

In [ ]:
import os
import torch
from d2l import torch as d2l

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

torch.set_printoptions(2)

help(torch.set_printoptions)

# 生成以每个像素为中心具有不同高宽度的锚框
def multibox_prior(data, sizes, ratios):
    # data.shape的最后两个元素为宽和高，第一个元素为通道数
    in_height, in_width = data.shape[-2:]
    # 数据对应的设备、锚框占比个数、锚框高宽比个数
    device, num_sizes, num_ratios = data.device, len(sizes), len(ratios)
    # 计算每个像素点对应的锚框数量
    boxes_per_pixel = (num_sizes + num_ratios - 1)
    # 将锚框占比列表转为张量并将其移动到指定设备
    size_tensor = torch.tensor(sizes, device=device)
    # 将宽高比列表转为张量并将其移动到指定设备
    ratio_tensor = torch.tensor(ratios, device=device)

    # 定义锚框中心偏移量
    offset_h, offset_w = 0.5, 0.5
    # 计算高度方向上的步长
    steps_h = 1.0 / in_height
    # 计算宽度方向上的步长
    steps_w = 1.0 / in_width

    # torch.arange(in_height, device=device)获得每一行像素
    # (torch.arange(in_height, device=device) + offset_h) 获得每一行像素的中心
    # (torch.arange(in_height, device=device) + offset_h) * steps_h 对每一行像素的中心坐标作归一化处理

    # 生成归一化的高度和宽度方向上的像素点中心坐标
    center_h = (torch.arange(in_height, device=device) + offset_h) * steps_h
    center_w = (torch.arange(in_width, device=device) + offset_w) * steps_w
    # 生成坐标网格
    shift_y, shift_x = torch.meshgrid(center_h, center_w)
    # 将坐标网格平铺为一维
    shift_y, shift_x = shift_y.reshape(-1), shift_x.reshape(-1)

    # 计算每个锚框的宽度和高度
    w = torch.cat((size_tensor * torch.sqrt(ratio_tensor[0]),
                   sizes[0] * torch.sqrt(ratio_tensor[1:]))) \
        * in_height / in_width
    h = torch.cat((size_tensor / torch.sqrt(ratio_tensor[0]),
                   sizes[0] / torch.sqrt(ratio_tensor[1:])))

    # 计算锚框的左上角和右下角坐标（相对于锚框中心的偏移量）
    anchor_manipulations = torch.stack((-w, -h, w, h)).T.repeat(in_height * in_width, 1) / 2

    # 计算所有锚框的中心坐标，每个像素对应boxes_per_pixel个锚框
    out_grid = torch.stack([shift_x, shift_y, shift_x, shift_y], dim=1).repeat_interleave(boxes_per_pixel, dim=0)

    # 通过中心坐标和偏移量计算所有锚框的左上角和右下角坐标
    output = out_grid + anchor_manipulations

    # 增加一个维度并返回结果
    return output.unsqueeze(0)


# 返回锚框变量Y的形状
img = d2l.plt.imread('01_Data/03_CatDog.jpg')
print("img.shape：", img.shape)  # 高561，宽72，3通道
h, w = img.shape[:2]
print(h, w)

X = torch.rand(size=(1, 3, h, w))  # 批量大小为1,3通道
Y = multibox_prior(X, sizes=[0.75, 0.5, 0.25], ratios=[1, 2, 0.5])  # 占图片sizes尺寸的大小、高宽比ratios尺寸大小的锚框
print(Y.shape)

# 访问以(250,250)为中心的第一个锚框
boxes = Y.reshape(h,w,5,4)  # 上面的sizes×sizes=3×3，3+3-1=5，故每个像素为中心生成五个锚框
boxes[250,250,0,:] # 以250×250为中心的第一个锚框的坐标


# 显示以图像中一个像素为中心的所有锚框
def show_bboxes(axes, bboxes, labels=None, colors=None):
    """显示所有边界框"""

    def _make_list(obj, default_values=None):
        # 如果obj为None，使用默认值；如果obj不是列表或元组，将其转换为列表
        if obj is None:
            obj = default_values
        elif not isinstance(obj, (list, tuple)):
            obj = [obj]
        return obj

    # 处理labels，确保其为列表形式
    labels = _make_list(labels)
    # 处理colors，确保其为列表形式
    colors = _make_list(colors, ['b', 'g', 'r', 'm', 'c'])
    # 遍历所有边界框
    for i, bbox in enumerate(bboxes):
        # 选择颜色
        color = colors[i % len(colors)]
        # 使用边界框和颜色生成矩形框
        rect = d2l.bbox_to_rect(bbox.detach().numpy(), color)
        # 在图像上添加矩形框
        axes.add_patch(rect)
        # 如果存在标签
        if labels and len(labels) > i:
            # 根据边界框的颜色选择标签的颜色
            text_color = 'k' if color == 'w' else 'w'
            # 在边界框上添加标签
            axes.text(rect.xy[0], rect.xy[1], labels[i], va='center',
                      ha='center', fontsize=9, color=text_color,
                      bbox=dict(facecolor=color, lw=0))


# 设置图像大小
d2l.set_figsize()
# 创建一个张量来缩放边界框的尺寸
bbox_scale = torch.tensor((w, h, w, h))
# 在图像上显示图像
fig = d2l.plt.imshow(img)
print("fig.axes：", fig.axes)
# 在生成锚框的时候是0-1的值，进行缩放的话就可以省点乘法运算，因为最后输出并不需要显示所有锚框，所以可能会更快一点
print("boxes[250,250,:,:]：", boxes[250, 250, :, :])
print("bbox_scale：", bbox_scale)
print("boxes[250,250,:,:] * bbox_scale：", boxes[250, 250, :, :] * bbox_scale)
show_bboxes(fig.axes, boxes[250, 250, :, :] * bbox_scale, ['s=0.75, r=1', 's=0.5, r=1', 's=0.25, r=1', 's=0.75,r=2',
                                                           's=0.75,r=0.5'])  # 画出以250×250像素为中心的不同高宽比的五个锚框

d2l.plt.show()